<a href="https://colab.research.google.com/github/fbeilstein/dbms/blob/master/DB_lecture_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* The primary job of any **database management system** (DBMS) is reliably storing data and making it available for users.

* We use DBs so we can concentrate on application logic instead of infrastructure.



The **storage engine** (or database engine) is a software component of a database management system responsible for storing, retrieving, and managing data in memory and on disk, designed to capture a persistent, long-term memory of each node.

* Databases respond to complex queries, storage engines look at
the data more granularly (simple API: create, update, delete, and retrieve records).

* Both keys and values can be arbitrary sequences of bytes with no prescribed form. Their sorting and representation semantics are defined in higher-level
subsystems. 

* DBMS can switch engines
  - MySQL (table-structured): InnoDB, MyISAM, and RocksDB, etc. 
  - MongoDB [(document-based](https://en.wikipedia.org/wiki/Document-oriented_database)): WiredTiger, In-Memory, MMAPv1, etc.

choose DB

* emulate workload
* [Yahoo! Cloud Serving Benchmark](https://en.wikipedia.org/wiki/YCSB#:~:text=The%20Yahoo!,of%20NoSQL%20database%20management%20systems.)
* TPC-C Benchmark (focuses on throughput and ACID)

**throughput**: the number of transactions the database system is able to process per minute.

ACID properties of transactions

* **Atomicity** All changes to data are performed as if they are a single operation. That is, all the changes are performed, or none of them are.
For example, in an application that transfers funds from one account to another, the atomicity property ensures that, if a debit is made successfully from one account, the corresponding credit is made to the other account.
* **Consistency** Data is in a consistent state when a transaction starts and when it ends. For example, in an application that transfers funds from one account to another, the consistency property ensures that the total value of funds in both the accounts is the same at the start and end of each transaction.
* **Isolation** The intermediate state of a transaction is invisible to other transactions. As a result, transactions that run concurrently appear to be serialized. For example, in an application that transfers funds from one account to another, the isolation property ensures that another transaction sees the transferred funds in one account or the other, but not in both, nor in neither.
* **Durability** After a transaction successfully completes, changes to data persist and are not undone, even in the event of a system failure.
For example, in an application that transfers funds from one account to another, the durability property ensures that the changes made to each account will not be reversed.

**DB architecture article**

Hellerstein, Joseph M., Michael Stonebraker, and James Hamilton. 2007. "Architecture of a Database System." Foundations and Trends in
Databases 1, no. 2 (February): 141-259. https://doi.org/10.1561/1900000002.

DBMS use a **client/server model**: DB instances (**nodes**) = servers, applications = **clients**.


Communication through the **transport subsystem**. Requests come in the form of some query language.

Query -> **query processor** (parse, interpret, validate) -> **query optimizer** (use statistics, ordering, choose access methods) -> **execution plan** -> **execution engine** (local queries are executed by the **storage engine**) -> Result

**Storage engine** components:

* **Transaction manager** This manager schedules transactions and ensures they cannot leave the database in a logically inconsistent state.
* **Lock manager** This manager locks on the database objects for the running transactions, ensuring that concurrent operations do not violate physical data integrity.
* **Access methods** (storage structures) These manage access and organizing data on disk. Access methods include heap files and storage structures such as B-Trees or LSM Trees.
* **Buffer manager** This manager caches data pages in memor.
* **Recovery manager** This manager maintains the operation log and restoring the system state in case of a failure.


Together, **transaction** and **lock** managers are responsible for concurrency control.

**In-memory database management systems** (sometimes called main memory DBMS) store data primarily in memory and use the disk for recovery and logging. 

**Disk-based DBMS** hold most of the data on disk and use memory for caching disk contents or as a temporary storage. 

During recovery, database contents can be restored from the backup and
logs. Log records are usually applied to backup in batches. After the batch of log records is processed, backup holds a database **snapshot** for a specific point in time, and log contents up to this point can be discarded. This process is called **checkpointing**.

Most database systems store a set of **data records**, consisting of **columns** and **rows** in **tables**. **Field** is an intersection of a column and a row: a single value of some type. Fields belonging to the same column usually have the same data type. A collection of values that belong logically to the same record (usually identified by the key) constitutes a row. Data is stored on disk: **row-** or **column-**wise. 

* **Row-oriented**: MySQL, PostgreSQL, and most of the traditional relational databases. 
* **Column-oriented**: MonetDB, C-Store (predecessor to Vertica),  Apache Kudu, ClickHouse.

**Column-oriented** databases should not be mixed up with **wide column stores**, where data is represented as a multidimensional map, columns are grouped into column families (usually storing data of the same type), and inside each column family, data is stored row-wise. Physical layout is somewhat different from the conceptual representation: column families are stored separately, but in each column family, the data belonging to the same key is stored together. This layout is best for storing data retrieved by a key or a sequence of keys. **Examples**: BigTable or HBase


Database systems do use files for storing the data, but instead of relying on filesystem hierarchies of directories and files for locating records, they compose files using implementation-specific formats. 

Database systems store **data records**, consisting of multiple fields, in tables, where each table is usually represented as a separate file. 

Each record in the table can be looked up using a **search key**. 

To locate a record, database systems use **indexes**: auxiliary data structures that allow it to efficiently locate data records without scanning an entire table on every access.

A database system usually separates **data files** and **index files**.

Files are partitioned into **pages**, which typically have the size of a single or multiple disk blocks. Pages can be organized as sequences of records or as  **slotted pages**.

Most modern storage systems do not delete data from pages explicitly, but use **tombstones**. 

Space occupied by the records **shadowed** by their updates or deletion markers is reclaimed during garbage collection

**Data files** (sometimes called primary files) 

* **index-organized tables (IOT)**
Records are not required to follow any particular order (they often use write order, easy data append),stored in the index itself in key order (fast range scans). It reduces disk seeks because we do not have to address a separate file.

* **heap-organized tables (heap files)**
require additional index structures, pointing to the locations where data records are stored, to make them searchable.
Store data records in the index itself.

* **hash-organized tables (hashed files)**
Records are stored in buckets (append order or sorted by key), and the hash value of the key determines which bucket a record belongs to.


When index is stored separately it shoud contain **data entries** with enough information to locate record in data file.


* **file offsets** (sometimes called **row locators**)
* locations of data records in the data file
*  **bucket IDs** in the case of hash files

**Index files**

An **index** is a structure that organizes data records on disk in a way that facilitates efficient retrieval operations. **Index files** are organized as specialized structures that map keys to locations in data files
* heap files -> records identified by these keys
* IOT -> primary keys

An index on a primary (data) file is called the **primary index**, other indexes are called **secondary**.
**Primary index files** hold a unique entry per search key, **secondary indexes** may hold several entries per search key. Multiple secondary indexes can point to the same record.

If the order of data records follows the search key order, this index is called **clustered** (also known as clustering), othewise **non-clustered** (**unclustered**). Data records in the clustered case are usually stored in the same file or in a clustered file, where the key order is preserved. 

Some engines have implicit primary key (e.g. MySQL InnoDB adds a new auto-increment column and fills in its values automatically).

Primary indexes are most often clustered, secondary -- non-clustered.


Two approaches to referencing


* Two indexes reference data entries directly from secondary index files.
* A secondary index goes through the indirection layer of a primary index to
locate the data entries

**Buffering, Immutability, and Ordering**

Storage structures have three common variables, you should choose:
* buffering or avoid using it 
* immutable or mutable files
* store values in order or out of order. 


**Buffering**
This defines whether or not the storage structure chooses to collect a certain
amount of data in memory before putting it on disk (avoidable buffering, not system-wide). 
**Methods**: Lazy B-Trees (B-Tree + in-memory buffers), two-component LSM Trees (different buffering + immutability)


**Mutability (or immutability)**
This defines whether or not the storage structure reads parts of the file, updates them, and writes the updated results at the same location in the file. Immutable structures are append-only.
**Methods**: copy-on-write (the updated page is written to the new location in the file, instead of its original location), LSM (immutable) and B-Trees (in-place update), Bw-Tree (immutable).


**Ordering**
This is defined as whether or not the data records are stored in the key order in the pages on disk. Ordering often defines whether or not we can efficiently
scan the range of records, not only locate the individual data records. Storing data out of order (most often, in insertion order) opens up for some write-time optimizations.
**Methods**: Bitcask, WiscKey (store data records directly in append-only files)

In [ ]:
# LSM Trees
https://www.youtube.com/watch?v=oUNjDHYFES8
# Write Amplification
https://en.wikipedia.org/wiki/Write_amplification
